In [1]:
import numpy as np
from cellpose_omni import models, core
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import os
from cellpose_omni import io
from cellpose_omni import io, transforms
from omnipose.utils import normalize99
from cellpose_omni import models
from cellpose_omni.models import MODEL_NAMES
from tkinter import filedialog
import scyjava
import tkinter as tk
from tkinter import filedialog
from pathlib import Path
import imagej
from jpype import JArray, JInt
import glob
from collections import defaultdict


MODEL_NAMES

2023-08-03 10:56:56,762 [INFO] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-08-03 10:56:56,763 [INFO] NumExpr defaulting to 8 threads.


/home/saka/pyvenv/omnipose/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['cyto',
 'nuclei',
 'cyto2',
 'bact_phase_cp',
 'bact_fluor_cp',
 'plant_cp',
 'worm_cp',
 'cyto2_omni',
 'bact_phase_omni',
 'bact_fluor_omni',
 'plant_omni',
 'worm_omni',
 'worm_bact_omni',
 'worm_high_res_omni']

In [2]:
scyjava.config.add_option('-Xmx30g')
start_dir = os.getcwd()
ij = imagej.init('sc.fiji:fiji', mode='interactive')
#ij = imagej.init('/home/saka/sw/local/fiji/2023', mode='interactive')
## Something about this init() function changes the current working directory.
ij.getApp().getInfo(True)
#ij.ui().showUI()
os.chdir(start_dir)
ij.getVersion() #This is to make sure ImageJ/Fiji opened properly. In case of error, it could display '2.9.0/inactive' instead of the full version number
rm = ij.RoiManager.getRoiManager()

In [5]:
root = tk.Tk()
root.withdraw()
directory_path = filedialog.askdirectory()
folder_list = os.listdir(directory_path)

In [8]:
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)
    file_pattern = os.path.join(folder_path, "*.tif")
    file_list = glob.glob(file_pattern)
    for file_path in file_list:
        imgs = io.imread(file_path)
        channels = [[0,0]]
        model_name = 'bact_phase_omni'
        model = models.CellposeModel(gpu=False, model_type=model_name)
        chans = [0,0]
        mask_threshold = -1
        verbose = 0 # turn on if you want to see more output 
        use_gpu = False #defined above
        transparency = True # transparency in flow output
        rescale=None # give this a number if you need to upscale or downscale your images
        omni = True # we can turn off Omnipose mask reconstruction, not advised 
        flow_threshold = 0 # default is .4, but only needed if there are spurious masks to clean up; slows down output
        niter = None # None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation 
        resample = True #whether or not to run dynamics on rescaled grid or original grid 
        cluster = True # use DBSCAN clustering
        augment = False # average the outputs from flipped (augmented) images; slower, usually not needed 
        tile = False # break up image into smaller parts then stitch together
        affinity_seg = 1 #new feature, stay tuned...
        masks, flows, styles = model.eval(imgs,
                                  channels=chans,
                                  rescale=rescale,
                                  mask_threshold=mask_threshold,
                                  transparency=transparency,
                                  flow_threshold=flow_threshold,
                                  niter=niter,
                                  omni=omni,
                                  cluster=cluster, 
                                  resample=resample,
                                  verbose=verbose, 
                                  affinity_seg=affinity_seg,
                                  tile=tile,
                                  augment=augment)
        io.save_masks(imgs, masks, flows, file_path, 
              png=False,
              tif=True, # whether to use PNG or TIF format
              suffix='', # suffix to add to files if needed 
              save_flows=False, # saves both RGB depiction as *_flows.png and the raw components as *_dP.tif
              save_outlines=False, # save outline images 
              dir_above=0, # save output in the image directory or in the directory above (at the level of the image directory)
              in_folders=True, # save output in folders (recommended)
              save_txt=True, # txt file for outlines in imageJ
              save_ncolor=False) # save ncolor version of masks for visualization and editing 

2023-08-03 11:06:32,781 [INFO] >>bact_phase_omni<< model set to be used
2023-08-03 11:06:32,782 [INFO] >>>> using CPU
2023-08-03 11:06:37,751 [INFO] >>bact_phase_omni<< model set to be used
2023-08-03 11:06:37,752 [INFO] >>>> using CPU
2023-08-03 11:06:41,599 [INFO] >>bact_phase_omni<< model set to be used
2023-08-03 11:06:41,600 [INFO] >>>> using CPU
2023-08-03 11:06:45,226 [INFO] >>bact_phase_omni<< model set to be used
2023-08-03 11:06:45,227 [INFO] >>>> using CPU
2023-08-03 11:06:49,369 [INFO] >>bact_phase_omni<< model set to be used
2023-08-03 11:06:49,370 [INFO] >>>> using CPU
2023-08-03 11:06:54,766 [INFO] >>bact_phase_omni<< model set to be used
2023-08-03 11:06:54,767 [INFO] >>>> using CPU
2023-08-03 11:06:58,918 [INFO] >>bact_phase_omni<< model set to be used
2023-08-03 11:06:58,919 [INFO] >>>> using CPU
2023-08-03 11:07:03,924 [INFO] >>bact_phase_omni<< model set to be used
2023-08-03 11:07:03,925 [INFO] >>>> using CPU
2023-08-03 11:07:09,268 [INFO] >>bact_phase_omni<< model

In [ ]:
root = tk.Tk()
root.withdraw()
filepath = filedialog.askopenfilename()

In [ ]:
image = ij.io().open(filepath)
#ij.ui().show(image)
imp = ij.py.to_imageplus(image)
ij.ui().show(imp)

In [ ]:
data_info = {}
for element in range(len(imp.dims)):
    name = imp.dims[element]
    data_info[name] = imp.shape[element]
    print(name)
num_frame = data_info['Z']

In [ ]:
file_prefix = "Image_T"
for timepoint in range(1, num_frame + 1):
    frame_number = f"{timepoint:04d}"
    #input_txt = Path(f"{directory_path}/txt_outlines/Experiment-1647-Split Scenes-01_z2/cellpose/frame_{frame_number}_cp_outlines.txt")
    input_txt = Path(f"{directory_path}/txt_outlines/{file_prefix}{frame_number}_Overlay_cp_outlines.txt")
    txt_fh = open(input_txt, 'r')
    for line in txt_fh:
            xy = line.rstrip().split(",")
            xy_coords = [int(element) for element in xy if element not in '']
            x_coords = [int(element) for element in xy[::2] if element not in '']
            y_coords = [int(element) for element in xy[1::2] if element not in '']
            xcoords_jint = JArray(JInt)(x_coords)
            ycoords_jint = JArray(JInt)(y_coords)
            polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
            roi_instance = scyjava.jimport('ij.gui.Roi')
            imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
            imp.setRoi(imported_polygon)
            rm.addRoi(imported_polygon)
            roi_count = rm.getCount()
            rm.select(roi_count - 1)
            time_set = imp.setZ(timepoint)   
            rm.runCommand('Update')
ij.py.run_macro("roiManager('Select All');")
f_name = os.path.basename(filepath)
f_name = os.path.splitext(f_name)[0]
rm.runCommand("Save", f"{directory_path}/" + f"{f_name}.zip")

In [ ]:
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)
    file_pattern = os.path.join(folder_path, "*.tif")
    file_list = glob.glob(file_pattern)
    for file_path in file_list:
        image_red = ij.io().open(file_pattern)
        imp = ij.py.to_imageplus(image_red)
        input_txt = Path(f"{folder_path}/Image_CH1_cp_outlines.txt")
        txt_fh = open(input_txt, 'r')
        for line in txt_fh:
            xy = line.rstrip().split(",")
            xy_coords = [int(element) for element in xy if element not in '']
            x_coords = [int(element) for element in xy[::2] if element not in '']
            y_coords = [int(element) for element in xy[1::2] if element not in '']
            xcoords_jint = JArray(JInt)(x_coords)
            ycoords_jint = JArray(JInt)(y_coords)
            polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
            roi_instance = scyjava.jimport('ij.gui.Roi')
            imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
            imp.setRoi(imported_polygon)
            rm.addRoi(imported_polygon)
        roi_count = rm.getCount()
        roi_index = JArray(JInt)(range(0, rm.getCount()))
        rm.setSelectedIndexes(roi_index)
        rm.runCommand("Save", f"{folder_path}/" + f"RoiSet.zip")
        rm.runCommand("Delete")

In [ ]:
# define parameters
mask_threshold = -1
verbose = 0 # turn on if you want to see more output 
use_gpu = use_GPU #defined above
transparency = True # transparency in flow output
rescale=None # give this a number if you need to upscale or downscale your images
omni = True # we can turn off Omnipose mask reconstruction, not advised 
flow_threshold = 0 # default is .4, but only needed if there are spurious masks to clean up; slows down output
niter = None # None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation 
resample = True #whether or not to run dynamics on rescaled grid or original grid 
cluster = True # use DBSCAN clustering
augment = False # average the outputs from flipped (augmented) images; slower, usually not needed 
tile = False # break up image into smaller parts then stitch together
affinity_seg = 1 #new feature, stay tuned...
masks, flows, styles = model.eval(imgs,
                                  channels=chans,
                                  rescale=rescale,
                                  mask_threshold=mask_threshold,
                                  transparency=transparency,
                                  flow_threshold=flow_threshold,
                                  niter=niter,
                                  omni=omni,
                                  cluster=cluster, 
                                  resample=resample,
                                  verbose=verbose, 
                                  affinity_seg=affinity_seg,
                                  tile=tile,
                                  augment=augment)

In [ ]:
io.save_masks(imgs, masks, flows, files, 
              png=False,
              tif=True, # whether to use PNG or TIF format
              suffix='', # suffix to add to files if needed 
              save_flows=False, # saves both RGB depiction as *_flows.png and the raw components as *_dP.tif
              save_outlines=False, # save outline images 
              dir_above=0, # save output in the image directory or in the directory above (at the level of the image directory)
              in_folders=True, # save output in folders (recommended)
              save_txt=True, # txt file for outlines in imageJ
              save_ncolor=False) # save ncolor version of masks for visualization and editing 